In [29]:
from policyengine_us import Microsimulation
from policyengine_core.reforms import Reform
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [30]:
# Define baseline takeup rates
baseline_takeup_rates = {
    "snap": 0.82,
    "medicaid": 0.93,
    "aca_ptc": 0.672,
}


In [31]:
# Map program names to reform parameter names
param_map = {
    "snap": "gov.usda.snap.takeup_rate",
    "medicaid": "gov.hhs.medicaid.takeup_rate",
    "aca_ptc": "gov.aca.takeup_rate"
}


In [32]:
# Create dataset path variable for reusability
dataset_path = "hf://policyengine/policyengine-us-data/enhanced_cps_2024.h5"


In [33]:
# Function to calculate enrollment or spending
def calculate_metric(sim, program, period=2026):
    values = sim.calculate(program, map_to="person", period=period)
    if program == "snap":
        # For SNAP, return enrollment (number of people)
        return (values > 0).sum() / 1e6  # in millions
    else:
        # For Medicaid and ACA, return spending
        return values.sum() / 1e9  # in billions


In [34]:
# Create takeup rate range from 0 to 1 with 0.05 steps
takeup_rates = np.arange(0, 0.102, 0.01)

# Store results for each program
results = {}

In [35]:
for program in ["snap", "medicaid", "aca_ptc"]:
    print(f"\nProcessing {program.upper()}...")
    
    baseline_takeup = baseline_takeup_rates[program]
    program_results = {
        'takeup_rate': [],
        'baseline_metric': [],
        'reformed_metric': [],
        'change_from_baseline': []
    }
    
    # Calculate baseline metric ONCE (using actual baseline takeup rate)
    baseline_reform = Reform.from_dict(
        {
            param_map[program]: {
                "2026-01-01.2100-12-31": baseline_takeup
            }
        },
        country_id="us"
    )
    baseline_sim = Microsimulation(dataset=dataset_path, reform=baseline_reform)
    baseline_metric = calculate_metric(baseline_sim, program)
    
    # Calculate metrics for each reformed takeup rate
    for rate in takeup_rates:
        # Create reform with current rate
        reform = Reform.from_dict(
            {
                param_map[program]: {
                    "2026-01-01.2100-12-31": rate
                }
            },
            country_id="us"
        )
        
        # Run reformed simulation
        reformed_sim = Microsimulation(dataset=dataset_path, reform=reform)
        reformed_metric = calculate_metric(reformed_sim, program)
        
        # Calculate change from baseline
        change = reformed_metric - baseline_metric
        
        # Store results
        program_results['takeup_rate'].append(rate)
        program_results['baseline_metric'].append(baseline_metric)
        program_results['reformed_metric'].append(reformed_metric)
        program_results['change_from_baseline'].append(change)
    
    results[program] = pd.DataFrame(program_results)


Processing SNAP...

Processing MEDICAID...

Processing ACA_PTC...


In [36]:
# Define program display names and metrics
programs = ["snap", "medicaid", "aca_ptc"]
titles = {"snap": "SNAP", "medicaid": "Medicaid", "aca_ptc": "ACA Premium Tax Credits"}
y_labels = {"snap": "Enrollment (millions)", "medicaid": "Spending (billions)", "aca_ptc": "Spending (billions)"}


In [37]:
# Create first figure: Baseline vs Reformed
fig1 = make_subplots(
    rows=1, cols=3,
    subplot_titles=[f'{titles[p]}: Baseline vs Reformed' for p in programs],
    horizontal_spacing=0.12
)

In [38]:

for idx, program in enumerate(programs):
    df = results[program]
    col = idx + 1
    
    # Add baseline line
    fig1.add_trace(
        go.Scatter(
            x=df['takeup_rate'],
            y=df['baseline_metric'],
            mode='lines',
            name='Baseline',
            line=dict(color='blue', width=2),
            showlegend=(idx == 0),
            hovertemplate='Takeup Rate: %{x:.2f}<br>' + y_labels[program] + ': %{y:.2f}<extra></extra>'
        ),
        row=1, col=col
    )
    
    # Add reformed line
    fig1.add_trace(
        go.Scatter(
            x=df['takeup_rate'],
            y=df['reformed_metric'],
            mode='lines',
            name='Reformed (rate - 0.05)',
            line=dict(color='red', width=2, dash='dash'),
            showlegend=(idx == 0),
            hovertemplate='Takeup Rate: %{x:.2f}<br>' + y_labels[program] + ': %{y:.2f}<extra></extra>'
        ),
        row=1, col=col
    )
    
    # Add vertical line for baseline takeup rate
    fig1.add_vline(
        x=baseline_takeup_rates[program],
        line_dash="dot",
        line_color="green",
        opacity=0.5,
        annotation_text=f"Current: {baseline_takeup_rates[program]:.2f}",
        annotation_position="top right",
        row=1, col=col
    )
    
    # Update axes
    fig1.update_xaxes(title_text="Takeup Rate", range=[0, 1], row=1, col=col)
    fig1.update_yaxes(title_text=y_labels[program], row=1, col=col)

# Update layout for first figure
fig1.update_layout(
    title_text="Takeup Rate Analysis: Baseline vs Reformed Scenarios",
    title_font_size=20,
    height=500,
    width=1400,
    hovermode='x unified'
)

In [39]:
# Create second figure: Change from Baseline
fig2 = make_subplots(
    rows=1, cols=3,
    subplot_titles=[f'{titles[p]}: Change from Baseline' for p in programs],
    horizontal_spacing=0.12
)

for idx, program in enumerate(programs):
    df = results[program]
    col = idx + 1
    
    # Add change line
    fig2.add_trace(
        go.Scatter(
            x=df['takeup_rate'],
            y=df['change_from_baseline'],
            mode='lines',
            name='Change from Baseline',
            line=dict(color='green', width=2),
            showlegend=(idx == 0),
            hovertemplate='Reform Takeup Rate: %{x:.2f}<br>Change: %{y:.2f}<extra></extra>'
        ),
        row=1, col=col
    )
    
    # Add horizontal line at y=0
    fig2.add_hline(
        y=0,
        line_color="black",
        opacity=0.3,
        row=1, col=col
    )
    
    # Add vertical line for baseline takeup rate
    fig2.add_vline(
        x=baseline_takeup_rates[program],
        line_dash="dot",
        line_color="green",
        opacity=0.5,
        annotation_text=f"Current: {baseline_takeup_rates[program]:.2f}",
        annotation_position="top right",
        row=1, col=col
    )
    
    # Update axes
    fig2.update_xaxes(title_text="Reform Takeup Rate", range=[0, 1], row=1, col=col)
    fig2.update_yaxes(title_text=f'Change in {y_labels[program].split()[0]} from Baseline', row=1, col=col)

# Update layout for second figure
fig2.update_layout(
    title_text="Impact of Reform: Change from Baseline",
    title_font_size=20,
    height=500,
    width=1400,
    hovermode='x unified'
)

In [40]:
# Show the figures
fig1.show()
fig2.show()

In [41]:
# Create summary statistics
print("\n" + "="*80)
print("SUMMARY STATISTICS")
print("="*80)

for program in programs:
    df = results[program]
    baseline_rate = baseline_takeup_rates[program]
    
    # Find the row corresponding to baseline takeup rate
    baseline_idx = int(baseline_rate * 100)
    if baseline_idx < len(df):
        baseline_value = df.iloc[baseline_idx]['baseline_metric']
        reformed_value = df.iloc[baseline_idx]['reformed_metric']
        change = df.iloc[baseline_idx]['change_from_baseline']
        
        metric_type = "enrollment (millions)" if program == "snap" else "spending (billions)"
        
        print(f"\n{titles[program]}:")
        print(f"  Baseline takeup rate: {baseline_rate:.2%}")
        print(f"  Baseline {metric_type}: {baseline_value:.2f}")
        print(f"  Reformed {metric_type} (at baseline rate): {reformed_value:.2f}")
        print(f"  Change: {change:+.2f} ({change/baseline_value*100:+.1f}%)")



SUMMARY STATISTICS


In [42]:
# Save results to CSV files
for program in programs:
    filename = f"{program}_takeup_analysis.csv"
    results[program].to_csv(filename, index=False)
    print(f"\nSaved {program} results to {filename}")



Saved snap results to snap_takeup_analysis.csv

Saved medicaid results to medicaid_takeup_analysis.csv

Saved aca_ptc results to aca_ptc_takeup_analysis.csv
